In [ ]:
# import libraries
import spotipy
import spotipy.util as util
import webbrowser
import json
import urllib.request
from nltk.corpus import stopwords
from nltk import pos_tag
import requests
import collections

In [ ]:
# the variable key now contains my API key
with open("news_key.txt", "r") as file:
    key = file.read()

In [ ]:
# this is the url for the news api
url = f"https://newsapi.org/v2/top-headlines?category=technology&apiKey={key}"

In [ ]:
# request is when im asking for data
request = urllib.request.Request(url)
# response is the answer i get for asking for data
response = urllib.request.urlopen(request)

In [ ]:
# i didn't remember what data type response is - so i printed it!
type(response)

In [ ]:
# converting the HTTPResponse object to a python dictionary
headlines = json.loads(response.read())

In [ ]:
headlines

In [ ]:
# make an empty list of my titles
# article_titles = []
# for headline in headlines['articles']:
#     article_titles.append(headline['title'])

article_titles = [x['title'] for x in headlines['articles']]

In [ ]:
article_titles

In [ ]:
# list of all possible POS tags from nltk: https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk
list_of_words = []
for headline in article_titles:
    tagged_sent = pos_tag(headline.split())    
    list_of_words.extend([word for word,pos in tagged_sent if pos == 'NNP' and word.lower() not in stopwords.words('english') and word.isalpha()])


In [ ]:
list_of_words

In [ ]:
word_counter = collections.Counter(list_of_words)
word_counter

In [ ]:
# if you want to remove duplicates in a list, convert to a dictionary and then back to a list
# because a dictionary does not allow duplicate keys and will remove them
list_of_words_no_repeat = list(dict.fromkeys(list_of_words))
list_of_words_no_repeat

In [ ]:
# open file with keys and set the path to your credentials JSON file
credentials = "spotify_keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [ ]:
api_tokens

In [ ]:
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]

In [ ]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [ ]:
# print out token
token

In [ ]:
# create my Spotify object
sp = spotipy.Spotify(auth=token)

In [ ]:
# start a list of songs for my headline playlist
songs_for_playlist = []
for keyword in word_counter.keys():
    # only take the 'top' words
    if word_counter[keyword] >= 2:
        # search for artists based on my keyword, only give me one artist
        searchResults = sp.search(q="artist:" + keyword, type="track", limit=1)
        # if the search returns anything
        if len(searchResults['tracks']['items']) > 0:
            songs_for_playlist.append(searchResults['tracks']['items'][0]['uri'])
    
print(songs_for_playlist)

In [ ]:
my_playlist = sp.user_playlist_create(user=username, name="Todays News", public=True,
                                      description="Songs for the news")
results = sp.user_playlist_add_tracks(username, my_playlist['id'], songs_for_playlist)
print(results)

In [ ]:
webbrowser.open(my_playlist['external_urls']['spotify'])